In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Test area #

In [48]:
df=pd.read_excel("facList.xlsx")
df=df.loc[:,['ทะเบียนโรงงาน ','ชื่อโรงงาน ']]
df['newCode']=""
df['facType']=""
df['ISIC']=""
df['TSIC']=""
df['desc']=""
df.head()

,ทะเบียนโรงงาน,ชื่อโรงงาน,newCode,facType,ISIC,TSIC,desc
0,ศ3-20(1)-1/34ชม,บริษัท เชียงใหม่ เบเวอเรช จำกัด,,,,,
1,ศ3-8(1)-4/36ชม,บริษัท สหปราจีนบุรีอุตสาหกรรมอาหาร จำกัด,,,,,
2,ศ3-8(1)-2/36ชม,บริษัท ช้างอวอดส์ (1959) จำกัด,,,,,
3,ศ3-8(1)-1/34ชม,บริษัท สันติภาพ (ฮั่วเพ้ง 1958) จำกัด,,,,,
4,3-58(1)-119/50ชม,บริษัท ไดมอนด์ วัสดุ จำกัด,,,,,


In [30]:
# Open Chrome
driver = webdriver.Chrome('chromedriver.exe')

# Open URL
driver.get('https://www.diw.go.th/hawk/content.php?mode=data1search')

In [49]:
facName="บริษัท เชียงใหม่ เบเวอเรช จำกัด"
facCode="ศ3-20(1)-1/34ชม"

In [50]:
driver.find_element_by_name('code').clear()
driver.find_element_by_name('code').send_keys(facCode)

In [51]:
driver.find_element_by_name('Name').clear()
driver.find_element_by_name('Name').send_keys(facName)

In [52]:
driver.find_element_by_name('Submit').click()

In [53]:
window_after = driver.window_handles[1]
#switch on to new child window
driver.switch_to.window(window_after)

In [54]:
soup = BeautifulSoup(driver.page_source,'html')
tables = soup.find_all('table')

In [55]:
#print(tables)

In [56]:
table=tables[0]

In [57]:
for tr in soup.find_all('tr')[1:2]:
    tds = tr.find_all('td')
    for i in tds:
        print(i.text.replace('\n','').replace('\r','').replace('\t','').strip())
        print("*-----------------------------------------------*")

บริษัท เชียงใหม่ เบเวอเรช จำกัด00500400125349 [ศ3-20(1)-1/34ชม]ประเภทโรงงาน 20(1),20(3)ISIC 1104 TSIC 11041
*-----------------------------------------------*
33/1 ม.5ถ.ซุปเปอร์ไฮเวย์ เชียงใหม่-ลำปาง ต.ยางเนิ้งอ.สารภี จ.เชียงใหม่   50140
*-----------------------------------------------*
บริษัท เชียงใหม่ เบเวอเรช จำกัด33/1ถ.ซุปเปอร์ไฮเวย์เชียงใหม่-ลำปาง ต.ยางเนิ้งอ.สารภี จ.เชียงใหม่   50140โทร. 053-420640-1
*-----------------------------------------------*
ผลิตโซดา และน้ำดื่มเครื่องจักร  6,072.28 HPเงินทุน 420,000,000 บาทคนงาน 141 คน
*-----------------------------------------------*


In [58]:
tr=soup.find_all('tr')[1]
td1=tr.find_all('td')[0]

In [59]:
td1_text=td1.text.replace('\n','').replace('\r','').replace('\t','').strip()

In [60]:
facCodeStart=td1_text.find(facName)+len(facName)

In [61]:
facCode=td1_text[facCodeStart:facCodeStart+14]

In [62]:
print(facCode)

00500400125349


In [63]:
tr=soup.find_all('tr')[1]
td2=tr.find_all('td')[3]
td2_text=td2.text.replace('\n','').replace('\r','').replace('\t','').strip()
print(td2_text)
print(td2_text[0:td2_text.find("เครื่องจักร")])

ผลิตโซดา และน้ำดื่มเครื่องจักร  6,072.28 HPเงินทุน 420,000,000 บาทคนงาน 141 คน
ผลิตโซดา และน้ำดื่ม


In [64]:
tr=soup.find_all('tr')[1]
td3=tr.find_all('td')[1]
td3_text=td3.text.replace('\n','').replace('\r','').replace('\t','').strip()
#print(td3_text)
if "โทร." in td3_text:
    print(td3_text[td3_text.find("โทร.")+4:].strip())

In [65]:
window_before = driver.window_handles[0]
#switch on to new child window
driver.switch_to.window(window_before)

# Deploy area #

In [70]:
df=pd.read_excel("facList.xlsx")
df=df.loc[:,['ทะเบียนโรงงาน ','ชื่อโรงงาน ']]
df[['newCode','facType','ISIC','TSIC','desc','tel']]="-"
#df=df.loc[0:10,:]
df.head()

,ทะเบียนโรงงาน,ชื่อโรงงาน,newCode,facType,ISIC,TSIC,desc,tel
0,ศ3-20(1)-1/34ชม,บริษัท เชียงใหม่ เบเวอเรช จำกัด,-,-,-,-,-,-
1,ศ3-8(1)-4/36ชม,บริษัท สหปราจีนบุรีอุตสาหกรรมอาหาร จำกัด,-,-,-,-,-,-
2,ศ3-8(1)-2/36ชม,บริษัท ช้างอวอดส์ (1959) จำกัด,-,-,-,-,-,-
3,ศ3-8(1)-1/34ชม,บริษัท สันติภาพ (ฮั่วเพ้ง 1958) จำกัด,-,-,-,-,-,-
4,3-58(1)-119/50ชม,บริษัท ไดมอนด์ วัสดุ จำกัด,-,-,-,-,-,-


In [71]:
# Open Chrome
driver = webdriver.Chrome('chromedriver.exe')

# Open URL
driver.get('https://www.diw.go.th/hawk/content.php?mode=data1search')

## Start loop

In [72]:
for i in df.index:
    #print(f"ทะเบียนโรงงานเก่า {df.iloc[i,0]} ชื่อโรงงาน {df.iloc[i,1]}")
    try :
        facName=df.iloc[i,1].strip()
        facCode=df.iloc[i,0].strip()
        driver.find_element_by_name('code').clear()
        driver.find_element_by_name('code').send_keys(facCode)
        driver.find_element_by_name('Name').clear()
        driver.find_element_by_name('Name').send_keys(facName)
        driver.find_element_by_name('Submit').click()
        window_after = driver.window_handles[1]
        driver.switch_to.window(window_after)

        soup = BeautifulSoup(driver.page_source,'html')
        tr=soup.find_all('tr')[1]
        td1=tr.find_all('td')[0]
        td1_text=td1.text.replace('\n','').replace('\r','').replace('\t','').strip()
        td2=tr.find_all('td')[3]
        td2_text=td2.text.replace('\n','').replace('\r','').replace('\t','').strip()
        td3=tr.find_all('td')[1]
        td3_text=td3.text.replace('\n','').replace('\r','').replace('\t','').strip()

        facCodeStart=td1_text.find(facName)+len(facName)
        facTypeStart=td1_text.find("ประเภทโรงงาน")+len("ประเภทโรงงาน")+1
        facIsicStart=td1_text.find("ISIC")+len("ISIC")+1
        facTsicStart=td1_text.find("TSIC")+len("ISIC")+1

        facCode=td1_text[facCodeStart:facCodeStart+14]
        facType=td1_text[facTypeStart:td1_text.find("ISIC")].strip()
        facIsic=td1_text[facIsicStart:facIsicStart+4]
        facTsic=td1_text[facTsicStart:facTsicStart+5]
        facDesc=td2_text[0:td2_text.find("เครื่องจักร")]
        
        if "โทร." in td3_text:
            facTel=td3_text[td3_text.find("โทร.")+4:].strip()
            df.loc[i,'tel']=facTel
            
        df.loc[i,'newCode']=facCode
        df.loc[i,'facType']=facType
        df.loc[i,'ISIC']=facIsic
        df.loc[i,'TSIC']=facTsic
        df.loc[i,'desc']=facDesc
        
        #print(f"ทะเบียนโรงงานใหม่ {facCode} ประเภทโรงงาน {facType} ISIC {facIsic} TSIC {facTsic}")
    except :
        print(f"ทะเบียนโรงงานเก่า {df.iloc[i,0]} ชื่อโรงงาน {df.iloc[i,1]}")
        print("Cannot search data")
        
    #print("--------------------------------------------")
    window_before = driver.window_handles[0]
    driver.switch_to.window(window_before)    

ทะเบียนโรงงานเก่า ศ3-8(2)-2/33ชร  ชื่อโรงงาน บริษัท แพนสตาร์ จำกัด 
Cannot search data
ทะเบียนโรงงานเก่า 3-22(3)-1/43นฐ  ชื่อโรงงาน บริษัท ที.ยู.ดับบลิว.เท็กซ์ไทล์ จำกัด 
Cannot search data
ทะเบียนโรงงานเก่า 3-51-2/50นฐ  ชื่อโรงงาน บริษัท สวซิ ซ์-วัน คอร์ปอเรชั่น จำกัด 
Cannot search data
ทะเบียนโรงงานเก่า 3-15(1)-4/51นพ  ชื่อโรงงาน บริษัท โกรเบสท์คอร์โพเรชั่น จำกัด 
Cannot search data
ทะเบียนโรงงานเก่า 3-9(6)-1/32อย  ชื่อโรงงาน บริษัท นครหลวง อินฟินนิทิเกรน จำกัด 
Cannot search data
ทะเบียนโรงงานเก่า 3-42(1)-1/36อย  ชื่อโรงงาน บริษัท พรวิไลอินเตอร์เนชั่นแนล กรุพ เทรดดิ้ง จำกัด 
Cannot search data
ทะเบียนโรงงานเก่า 3-53(4)-31/47อย  ชื่อโรงงาน บริษัท วิงฟง แอดฮีซีฟ แมนูแฟคเจอริ่ง (ประเทศไทย) จำกัด 
Cannot search data
ทะเบียนโรงงานเก่า จ3-1-1/42พย  ชื่อโรงงาน สถานีบ่มใบยาดอนไชยป่าแขม 
Cannot search data
ทะเบียนโรงงานเก่า 3-1-1/31พย  ชื่อโรงงาน ถานีบ่มใบยาสูบสันกลาง 
Cannot search data
ทะเบียนโรงงานเก่า ศ3-10(3)-1/34รบ  ชื่อโรงงาน บริษัท นาเชา (ประเทศไทย) จำกัด 
Cannot search data
ทะเบียน

In [75]:
df

,ทะเบียนโรงงาน,ชื่อโรงงาน,newCode,facType,ISIC,TSIC,desc,tel
0,ศ3-20(1)-1/34ชม,บริษัท เชียงใหม่ เบเวอเรช จำกัด,00500400125349,"20(1),20(3)",1104,11041,ผลิตโซดา และน้ำดื่ม,-
1,ศ3-8(1)-4/36ชม,บริษัท สหปราจีนบุรีอุตสาหกรรมอาหาร จำกัด,00500000425362,8(1),1030,10302,ทำผัก-ผลไม้ น้ำผัก-ผลไม้ บรรจุกระป๋อง,053-884194-5
2,ศ3-8(1)-2/36ชม,บริษัท ช้างอวอดส์ (1959) จำกัด,00500000225366,8(1),1030,10302,ทำอาหารกระป๋องจากผัก พืชหรือผลไม้และดองผักพืชห...,0-5388-4857-62
3,ศ3-8(1)-1/34ชม,บริษัท สันติภาพ (ฮั่วเพ้ง 1958) จำกัด,00500000125343,8(1),1030,10302,ผลิตผักและผลไม้บรรจุกระป๋องสำเร็จรูป,297984
4,3-58(1)-119/50ชม,บริษัท ไดมอนด์ วัสดุ จำกัด,10500011925507,58(1),2395,23953,ผลิตอิฐมวลเบา อบไอน้ำ,053-849434-8
...,...,...,...,...,...,...,...,...
280,3-102-2/57สบ,บริษัท ทีพีไอ โพลีน จำกัด (มหาชน),10190000225570,88,3530,35302,ผลิตไอน้ำ ขนาด 35 ตันต่อชั่วโมง และผลิตพลังงาน...,036-6703670-5
281,3-102-3/58สบ,บริษัท ทีพีไอ โพลีน เพาเวอร์ จำกัด(มหาชน),10190000325586,102,3530,35302,ผลิตพลังงานไอน้ำ,-
282,3-58(1)-1/38สห,บริษัท อินทรี ซุปเปอร์บล๊อก จำกัด,10170100125383,58(1),2395,23953,ผลิตคอนกรีตมวลเบา,036-535370
283,3-58(1)-1/39สห,บริษัท แอโรกรีต (ประเทศไทย) จำกัด,10170100125391,58(1),2395,23953,ทำผลิตภัณฑ์คอนกรีตผสมยิบซัม เช่น ผนังคอนกรีตเบา,-


In [76]:
df.to_csv("scrapData.csv")